# Replicating the M4 competition using sktime

Resources: 
* [Github repo of the M4 competition](https://github.com/M4Competition/M4-methods)

## Preliminaries

In [15]:
import numpy as np
import pandas as pd
import os
from sklearn.utils.validation import check_consistent_length
from losses import smape_loss
from losses import mase_loss

### Set paths

In [3]:
repodir = "/Users/mloning/Documents/Research/python_methods/m4-methods/"
datadir = os.path.join(repodir, "Dataset")
traindir = os.path.join(datadir, 'Train')
testdir = os.path.join(datadir, 'Test')
savedir = os.path.join(repodir, "predictions")

assert os.path.exists(datadir)
assert os.path.exists(traindir)
assert os.path.exists(testdir)
assert os.path.exists(savedir)

### Load results from M4 competition for comparison

In [4]:
m4_results = pd.read_excel(os.path.join(repodir, 'Evaluation and Ranks.xlsx'), 
                        sheet_name='Point Forecasts-Frequency',
                        header=[0, 1]).dropna(axis=0)

mase = m4_results.loc[:, ['Method', 'MASE']]
mase.columns = mase.columns.droplevel()
mase = mase.set_index('User ID')

smape = m4_results.loc[:, ['Method', 'sMAPE']]
smape.columns = smape.columns.droplevel()
smape = smape.set_index('User ID')

print(mase.shape, smape.shape)

(59, 7) (59, 7)


### Import meta data

In [5]:
import pandas as pd
info = pd.read_csv(os.path.join(datadir, 'M4-info.csv'))

In [6]:
info.head()

,M4id,category,Frequency,Horizon,SP,StartingDate
0,Y1,Macro,1,6,Yearly,01-01-79 12:00
1,Y2,Macro,1,6,Yearly,01-01-79 12:00
2,Y3,Macro,1,6,Yearly,01-01-79 12:00
3,Y4,Macro,1,6,Yearly,01-01-79 12:00
4,Y5,Macro,1,6,Yearly,01-01-79 12:00


In [7]:
n_datasets = info.shape[0]
print(n_datasets)

100000


In [8]:
info.category.value_counts()

Micro          25121
Finance        24534
Macro          19402
Industry       18798
Demographic     8708
Other           3437
Name: category, dtype: int64

In [9]:
info.SP.value_counts()

Monthly      48000
Quarterly    24000
Yearly       23000
Daily         4227
Hourly         414
Weekly         359
Name: SP, dtype: int64

In [10]:
# dictionary of forecasting horizons
fhs = info.set_index('SP').Horizon.to_dict()
fhs

{'Yearly': 6,
 'Quarterly': 8,
 'Monthly': 18,
 'Weekly': 13,
 'Daily': 14,
 'Hourly': 48}

In [11]:
# dictionary of frequencies
freqs = info.set_index('SP').Frequency.to_dict()
freqs

{'Yearly': 1,
 'Quarterly': 4,
 'Monthly': 12,
 'Weekly': 1,
 'Daily': 1,
 'Hourly': 24}

In [12]:
# look up individual time series
info.loc[info.loc[:, 'M4id'] == 'M4', :]

,M4id,category,Frequency,Horizon,SP,StartingDate
47003,M4,Macro,12,18,Monthly,01-09-08 12:00


### Load data files

In [13]:
files = os.listdir(traindir)
files

['Weekly-train.csv',
 'Daily-train.csv',
 'Hourly-train.csv',
 'Monthly-train.csv',
 'Yearly-train.csv',
 'Quarterly-train.csv']

In [14]:
keys = [f.split('-')[0] for f in files]
keys

['Weekly', 'Daily', 'Hourly', 'Monthly', 'Yearly', 'Quarterly']

In [120]:
sl = []
for key in keys:
    alltrain = pd.read_csv(os.path.join(traindir, f'{key}-train.csv'),
                           index_col=0)
    sl.append(alltrain.notna().sum(axis=1).describe())

series_lengths = pd.concat(sl, axis=1).T
series_lengths.index = keys
series_lengths['total_n_obs'] = series_lengths['count'] * series_lengths['mean']
series_lengths['fh'] = pd.Series(fhs)
series_lengths.sort_values('total_n_obs', ascending=False)

,count,mean,std,min,25%,50%,75%,max,total_n_obs,fh
Monthly,48000.0,216.300229,137.406295,42.0,82.0,202.0,306.0,2794.0,10382411.0,18
Daily,4227.0,2357.383014,1756.568917,93.0,323.0,2940.0,4197.0,9919.0,9964658.0,14
Quarterly,24000.0,92.254500,51.129507,16.0,62.0,88.0,115.0,866.0,2214108.0,8
Yearly,23000.0,31.324261,24.523966,13.0,20.0,29.0,40.0,835.0,720458.0,6
Weekly,359.0,1022.038997,707.148455,80.0,379.0,934.0,1603.0,2597.0,366912.0,13
Hourly,414.0,853.864734,127.945362,700.0,700.0,960.0,960.0,960.0,353500.0,48


### Compare results

In [41]:
assert n_datasets == info.SP.value_counts()[key]

selected_methods = ['Naive', 'Naive2', 'SES', 'Holt' ,'Damped']
n_selected_methods = len(selected_methods)
sig = key[0] 

replicated = np.zeros((n_selected_methods, 2))
original = np.zeros((n_selected_methods, 2))
for j, method in enumerate(selected_methods):
    results = np.zeros((n_datasets, 2))
    for i in range(n_datasets):
        fname = f"{method}_{sig}{i}_y_pred.txt"
        y_pred = np.loadtxt(os.path.join(savedir, fname))
        y_train = alltrain.iloc[i, :].dropna().reset_index(drop=True)
        y_test = alltest.iloc[i, :].dropna().reset_index(drop=True)
        results[i, 0] = mase_loss(y_test, y_pred, y_train, freq=freq)
        results[i, 1] = smape_loss(y_test, y_pred)

    replicated[j, :] = results.mean(axis=0)
    original[j, :] = np.hstack([mase.loc[method, key], smape.loc[method, key]])

replicated = pd.DataFrame(replicated, index=selected_methods, columns=['mase', 'smape'])
original = pd.DataFrame(original, index=selected_methods, columns=['mase', 'smape'])

In [17]:
def percentage_difference(x, y):
    return (x - y) / x * 100

In [64]:
perc_diff = ((original - replicated.round(3)) / original * 100).round(3)
perc_diff

,mase,smape
Naive,0.000,0.000
Naive2,0.000,0.000
SES,0.037,0.011
Holt,1.612,3.358
Damped,-2.995,-2.628
